In [1]:
import pickle
import numpy as np
import random
import glob
import matplotlib.pylab as plt
import numpy as np
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as func
from torchvision.models import resnet18
from torchvision import transforms, models
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import entropy
from getData import *
from trainer import *
from getModel import *
#import torchprofile

import time
# Seed for reproducibility
SEED = 25
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# **Load Dataset**

In [2]:
dataset = TrainValDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size= 32, shuffle = True, num_workers= 0)

val_loader = DataLoader(val_dataset, batch_size= 32, shuffle = False, num_workers= 0)

test_dataset = TestDataset()

test_loader = DataLoader(test_dataset, batch_size= 32, shuffle = False, num_workers= 0)


['Training\\Fire', 'Training\\No_Fire']
['Test\\Fire', 'Test\\No_Fire']


# **Baseline Model**

In [3]:
NUM_EPOCHS = 5

In [4]:
blNet = blModel().to(device)
optimizer_bl = optim.Adam(blNet.parameters(), lr=0.0001, weight_decay=5e-4)
scheduler = None
criterion_bl = nn.CrossEntropyLoss()

In [5]:
bl_titleToday = "./Models/best_bl_0319.pth"
BL = blHandler(blNet, criterion_bl, optimizer_bl, device=device, scheduler=scheduler, num_epochs=NUM_EPOCHS, bestPath=bl_titleToday)
blNet, blHistory = BL.train(train_loader, val_loader)

epoch 1 --> trainLoss: 0.345, trainAcc: 0.968, validLoss: 0.341, validAcc: 0.973
Model Saved!
epoch 2 --> trainLoss: 0.336, trainAcc: 0.978, validLoss: 0.338, validAcc: 0.976
Model Saved!
epoch 3 --> trainLoss: 0.334, trainAcc: 0.980, validLoss: 0.337, validAcc: 0.976
Model Saved!
epoch 4 --> trainLoss: 0.333, trainAcc: 0.980, validLoss: 0.337, validAcc: 0.976
epoch 5 --> trainLoss: 0.333, trainAcc: 0.980, validLoss: 0.335, validAcc: 0.978
Model Saved!


In [6]:
bl_titleToday = "./Models/best_bl_0319.pth"
blNet.load_state_dict(torch.load("./Models/best_bl_0319.pth"))
BL = blHandler(blNet, criterion_bl, optimizer_bl, device=device, scheduler=scheduler, num_epochs=NUM_EPOCHS, bestPath=bl_titleToday)
predicted, acc = BL.infer(test_loader)

In [7]:
print(acc)

0.6162037037037037


# **EEM Model**